In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [4]:
import os
import cv2
import numpy as np
import pickle
from tqdm import tqdm

import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

/home/c.marzahl@de.eu.local/anaconda3/envs/fastai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
properties = pickle.load(open( "data_2f.pkl", "rb" ))
log_dir = '/data/Datasets/WhiteBloodCancer/projection/'
experiment_name = "2_features"
image_size = (128,128)
metadata_file = open(os.path.join(log_dir, '{}.tsv'.format(experiment_name)), 'w')

In [6]:
sprite_image_shape = 8192
sprite_image = np.zeros(shape=(sprite_image_shape, sprite_image_shape, 3), dtype=np.int8)
total_columns = int(np.floor(sprite_image_shape / image_size[0]))
total_rows = int(np.floor(sprite_image_shape / image_size[1]))

In [7]:
x, y, idx = 0, 0, 0
features = {'shape_features': [],
            'cnn_avg_features': [],
            'cnn_max_features': [],
            'intensity_features': [],
            'cnn_flatt_features': []}

In [8]:
metadata_file.write('\t'.join(['Class', 'ID', 'Patient']) + '\n')
for key in tqdm(list(properties)[:total_columns*total_rows]):
    item = properties[key]
    line = '\t'.join(map(str, [item['Class'], key, item['Patient']])) + '\n'
    metadata_file.write(line)

    img = cv2.cvtColor(cv2.resize(cv2.imread(str(item['Path'])), image_size), cv2.COLOR_BGR2RGB)

    # extract width und height
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]
    features['shape_features'].append([cmax-cmin, rmax-rmin, len(np.nonzero(img[:, :, 0])[0])])

    red = img[:, :, 0]
    green = img[:, :, 1]
    blue = img[:, :, 2]
    features['intensity_features'].append([np.mean(red[red > 0]),
                                           np.mean(green[green > 0]),
                                           np.mean(blue[blue > 0]),
                                           np.std(red[red > 0]),
                                           np.std(green[green > 0]),
                                           np.std(blue[blue > 0])])

    features['cnn_flatt_features'].append(item['features'].flatten())
    features['cnn_avg_features'].append(np.array([np.max(axis) for axis in item['features']]).flatten())
    features['cnn_max_features'].append(np.array([np.mean(axis) for axis in item['features']]).flatten())


    sprite_image[y * img.shape[0]:y * img.shape[0] + img.shape[0], x * img.shape[1]:x * img.shape[1] + img.shape[1]] = img
    x = idx % total_rows
    y = idx // total_columns
    idx += 1

100%|██████████| 1066/1066 [00:01<00:00, 862.73it/s]


In [9]:
features['cnn_max_features'] = np.array(features['cnn_max_features'])
features['cnn_avg_features'] = np.array(features['cnn_avg_features'])
features['cnn_flatt_features'] = np.array(features['cnn_flatt_features'])

cv2.imwrite(os.path.join(log_dir, '{}.png'.format(experiment_name)), sprite_image)
metadata_file.close()

In [10]:
feature_vectors = [tf.Variable(features[item], name=item) for item in features]

with tf.Session() as sess:
    saver = tf.train.Saver(feature_vectors)

    sess.run(tf.global_variables_initializer())
    saver.save(sess, os.path.join(log_dir, '{}.ckpt'.format(experiment_name)))

    config = projector.ProjectorConfig()

    for features in feature_vectors:
        # One can add multiple embeddings.
        embedding = config.embeddings.add()
        embedding.tensor_name = features.name
        # Link this tensor to its metadata file (e.g. labels).
        embedding.metadata_path = '{}.tsv'.format(experiment_name)
        if sprite_image is not None:
            embedding.sprite.image_path = '{}.png'.format(experiment_name)
            embedding.sprite.single_image_dim.extend(image_size)

    # Saves a config file that TensorBoard will read during startup.
    projector.visualize_embeddings(tf.summary.FileWriter(log_dir), config)